In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.naive_bayes import GaussianNB
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

In [ ]:
digits = load_digits() # 手写数据集
X, y = digits.data, digits.target
x_train, x_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state=420)



In [ ]:
x_train.shape #(1257, 64)
x_valid.shape #(540, 64)
np.unique(y_train) # 是个特征
# 多分类问题，类别十个


In [ ]:
gnb = GaussianNB().fit(x_train, y_train)
acc_score = gnb.score(x_valid, y_valid) # 返回预测的精确性accuracy
y_pred = gnb.predict(x_valid)
y_pred # 返回所有分类
prob = gnb.predict_proba(x_valid)
prob # 每一列对应的标签概率
# 使用混淆矩阵来查看结果
from sklearn.metrics import confusion_matrix as CM
CM(y_valid, y_pred) # 模型结果还不错

In [ ]:
# 高斯朴素贝叶斯,具体的各种形状的分类数据比较可以参考决策树和svm
# 月亮型，二分型， 环形...，比较适合线性可分的数据,但是非线性也能做，不是纯粹的线性的，主要问题是需要不同的特征之间需要时独立的
# 使用sklearn中自带的绘制学习曲线的类learning_curve，在这个类下执行交叉验证并从中获得不同样本量下的训练和测试准确度
import numpy as np
import matplotlib.pyplot as plt
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.tree import DecisionTreeClassifier as DTC
from sklearn.linear_model import LogisticRegression as LR
from sklearn.datasets import load_digits
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit
from time import time
import datetime

In [ ]:
clf = GaussianNB()
cv = ShuffleSplit(n_splits=50 # 把数据分为多少份
                    , test_size=0.2 # 20% * 50份的数据会被作为测试集
                    , random_state=0 # 分交叉验证的份数时进行随机抽样的模式
                    )
train_sizes, train_scores, test_scores = learning_curve(clf, X, y, cv=cv,n_jobs=4)              

In [ ]:
train_sizes #[ 143,  467,  790, 1113, 1437] 每次分训练集和测试集建模之后，训练集上的样本数量
train_scores # 训练集上的分数
train_scores.shape #(5, 50)
test_scores.shape #5个取值之下进行50次交叉验证的结果

In [ ]:
# 输入我的分类器，一次画出所有的学习曲线
def plot_learning_curve(estimator, title, X, y,
                        ax, # 选择子图
                        ylim=None, #设置纵坐标的取值范围
                        cv=None, #交叉验证
                        n_jobs=None#设定要素使用的线程
                        ):
    train_sizes, train_scores, test_scores = learning_curve(estimator, X, y, cv=cv,n_jobs=n_jobs)
    ax.set_title(title)
    if ylim is not None:
        ax.set_ylim(*ylim) # 保持y轴的量纲相同，使得对比时更加直观
    ax.set_xlabel("training example")
    ax.set_ylabel("score")
    ax.grid() # 显示网格作为背景
    ax.plot(train_sizes, np.mean(train_scores, axis=1), "o-", color="r", label="training score")
    ax.plot(train_sizes, np.mean(test_scores, axis=1), "o-", color="g", label= "test score")
    ax.legend(loc="best")
    return ax


In [ ]:
title = ["naive bayes", "decisiontree", "svm, rbf kernel", "ramdomforest", "logistic"]
model = [GaussianNB(), DTC(), SVC(gamma=0.001), RFC(n_estimators=50),LR(C=.1, solver="lbfgs")]
cv = ShuffleSplit(n_splits=50, test_size=0.2, random_state=0)

In [ ]:
fig, axes = plt.subplots(1,5,figsize=(30, 6))
for ind, title_, estimator in zip(range(len(title)), title, model):
    times = time()
    plot_learning_curve(estimator, title_, X, y,ax=axes[ind],ylim=[0.7, 1.05],n_jobs=4,cv=cv)
    print("{}:{}".format(title_, datetime.datetime.fromtimestamp(time()-times).strftime("%M:%S:%f")))
plt.show()
# 样本很大的时候，准确率贝叶斯会下降，适合小样本数据,
#逻辑回归比贝叶斯效果好，但是维度高会运行非常慢
#这两个算法恢复的是一个概率

In [ ]:
# 贝叶斯评估指标
# 在二分类问题中，一般会运用布利尔指标，是一种loss
from sklearn.metrics import brier_score_loss
# 注意第一个参数是真实标签，第二个参数是预测出来的
# 二分类下，predict_proba会返回两列，svc的decision_function会返回一列
brier_score_loss(y_valid,prob[:,1],pos_label=1)
# 我们的pos_label和prob中的索引一致,就可以查看这个类别下的布利尔分数是多少

# 多分类中
for i in range(0, 10):
    print(brier_score_loss(y_valid,prob[:,i],pos_label=i))

# 只能返回选择的类的布利尔

In [ ]:
# 返回概率的模型都可以运用布利尔分数
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression as LR
logi = LR(C=.1, solver="lbfgs", max_iter=3000, multi_class="auto").fit(x_train,y_train)
svc = SVC(kernel="linear", gamma=1).fit(x_train, y_train)
brier_score_loss(y_valid, logi.predict_proba(x_valid)[:, 1],pos_label=1)
# 由于svc的置信度并不是概率，为了可比性，我们需要将svc的置信度距离归一化，压缩到0-1之间
svc_prob = (svc.decision_function(x_valid) - svc.decision_function(x_valid).min()) / (svc.decision_function(x_valid).max() - svc.decision_function(x_valid).min())
brier_score_loss(y_valid, svc_prob[:, 1],pos_label=1)
# 支持向量机最差，因为本身就不是返回概率的模型
# 接下来进行一下可视化
import pandas as pd
name = ["bayes", "logistic", "svc"]
color = ["red", "black", "orange"]
df = pd.DataFrame(index=range(10), columns=name)
for i in range(10):
    df.loc[i, name[0]] = brier_score_loss(y_valid, prob[:, i], pos_label=i)
    df.loc[i, name[1]] = brier_score_loss(y_valid, logi.predict_proba(x_valid)[:, i], pos_label=i)
    df.loc[i, name[2]] = brier_score_loss(y_valid, svc_prob[:, i], pos_label=i)
for i in range(df.shape[1]):
    plt.plot(range(10), df.iloc[:, i],c = color[i])
plt.legend()
plt.show()
# 逻辑回归相对最好，支持向量机就很差


In [ ]:
# 指标对数似然函数log loss
# 可以应用于多酚类
# y_pred是给的概率
from sklearn.metrics import log_loss
log_loss(y_valid, prob) #2.4725653911460683
log_loss(y_valid, logi.predict_proba(x_valid)) #0.10398581573876683
log_loss(y_valid, svc_prob) #1.625556312147472
# 在这个评价指标下，支持向量机比贝叶斯好了，因为支持向量机本身就是最优化损失函数，所以会在损失评价下，效果很好
# 现实下log_loss进行评价，因为贝叶斯基本用不上
#对数似然看多分类，对比多个模型，可解释性差，最优化指向svc，逻辑回归，数学上概率无法取到1或0，只能接近
#布利尔看二分类，衡量单一模型，可解释性好，指向朴素贝叶斯，概率可以为0， 1，比如树或者随机森林可以用布利尔
# 若贝叶斯算法效果不好，还不换模型，我们要调节校准程度，来强行调整，下面介绍


In [ ]:
# 可靠性曲线reliability curve
#是以预测概率为横坐标，真实标签为纵坐标的曲线，因此模型或者算法的概率校准曲线越靠近对角线越好。通常应用于二分类问题
import numpy as np
import matplotlib.pyplot as plt
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.tree import DecisionTreeClassifier as DTC
from sklearn.linear_model import LogisticRegression as LR
from sklearn.datasets import load_digits
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit
from time import time
from sklearn.datasets import make_classification as mc
from sklearn.model_selection import train_test_split
import datetime

In [ ]:
X, y = mc(n_samples=100000, n_features=20 #总共20个特征
            , n_classes=2#标签分为两类
            ,n_informative=2# 器中两个代表较多信息
            ,n_redundant=10 # 是个都是冗余特征
            ,random_state=42)
# 样本量足够大，因此使用1%的样本作为训练集
x_train,x_valid,y_train,y_valid = train_test_split(X, y,test_size=0.99,random_state=42)
x_train
#######
gnb = GaussianNB()
gnb.fit(x_train, y_train)
y_pred = gnb.predict(x_valid)
prob_pos = gnb.predict_proba(x_valid)[:,1]# 我们的预测概率 - 横坐标
# 我们的真实标签 - 纵坐标
# 利用字典创建dataframe
df = pd.DataFrame({"ytrue":y_valid[:500],"probability":prob_pos[:500]})
df
df = df.sort_values("probability")
# 恢复索引
df.index = range(df.shape[0])
fig = plt.figure()# 画布
ax1 = plt.subplot() # 建立一个子图
ax1.plot([0, 1],[0, 1], "k:", label="perfectly calibrated")# 做一条对角线来进行对比
ax1.plot(df["probability"],df["ytrue"], "s-")#, label="%s (%1.3f)" % ("bayes", clf_score))
ax1.set_ylabel("true label")
ax1.set_xlabel("predicted probability")
ax1.set_ylim([-0.05, 1.06])
ax1.legend()
plt.show()


# 这个图就没什么意义了，0 1 之间一直跳，我们应该找真实的概率是多少，但不能求，所以用另一种方式找
# 一个简单的做法是，将数据进行分箱，然后规定每个箱子中真实的少数类所占的比例为这个箱子上的真是概率trueproba,这个箱子中预测概率的均值为这个箱子的预测概率
#predproba，然后以trueproba为纵坐标，predproba为横坐标，来绘制我们的额可靠性曲线
# sklearn中可以通过绘制可靠性曲线的类calibrationcurve实现
# 参数normalize对输入归一化，y_pred可以输入正类别下的概率或者置信度




In [ ]:
from sklearn.calibration import calibration_curve
 # 从类calibirtion_curve中获取横坐标和纵坐标
trueproba, predproba = calibration_curve(y_valid, prob_pos, n_bins=10)

fig = plt.figure()# 画布
ax1 = plt.subplot() # 建立一个子图
ax1.plot([0, 1],[0, 1], "k:", label="perfectly calibrated")# 做一条对角线来进行对比
ax1.plot(predproba, trueproba, "s-")#, label="%s (%1.3f)" % ("bayes", clf_score))
ax1.set_ylabel("true label")
ax1.set_xlabel("predicted probability")
ax1.set_ylim([-0.05, 1.06])
ax1.legend()
plt.show()





In [ ]:
#探索不同箱子数对结果的影响
fig, axes = plt.subplots(1, 3, figsize=(18, 4))
for ind, i in enumerate([3, 10, 100]):
    ax = axes[ind]
    ax.plot([0, 1],[0, 1], "k:", label="perfectly calibrated")# 做一条对角线来进行对比
    trueproba, predproba = calibration_curve(y_valid, prob_pos, n_bins=i)
    ax.plot(predproba, trueproba, "s-")#, label="%s (%1.3f)" % ("bayes", clf_score))
    ax1.set_ylabel("true label")
    ax1.set_xlabel("predicted probability")
    ax1.set_ylim([-0.05, 1.06])
    ax1.legend()
plt.show()

In [ ]:
name = ["gaussianbayes", "logistic", "svc"]
gnb = GaussianNB()
logi = LR(C=.1, solver="lbfgs", max_iter=3000, multi_class="auto")
svc = SVC(kernel="linear", gamma=1)
fig, ax1 = plt.subplots(figsize=(8, 6))
ax1.plot([0, 1], [0, 1], "k:", label="perfectly calibrated")
for clf, name_ in zip([gnb, logi, svc], name):
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_valid)
    # hasattr(obj.name): 查看一个类obj中是否存在名字为name的接口，存在则返回true
    if hasattr(clf, "predict_proba"):
        prob_pos = clf.predict_proba(x_valid)[:, 1]
    else: # use decision function 
        prob_pos = clf.decision_function(x_valid)
        prob_pos = (prob_pos - prob_pos.min()) / (prob_pos.max() - prob_pos.min())

        # 返回布利尔分数
    clf_score = brier_score_loss(y_valid, prob_pos, pos_label=y.max())
    trueproba, predproba = calibration_curve(y_valid, prob_pos, n_bins=10)
    ax1.plot(predproba, trueproba, "s-", label="%s (%1.3f)" % (name_, clf_score))

ax1.set_ylabel("")
ax1.set_xlabel("")
ax1.set_ylim([-.05, 1.05])
ax1.legend()
fig.show()

# 逻辑回归天生完美的返回概率的模型

# 支持向量机和高斯贝叶斯都是比较糟糕的结果
# 对于贝叶斯呈现出来sigmoid函数的镜像的情况，说明数据集中的特征不是相互条件独立的
# 支持向量机是典型的置信度不足的分类器
# 大量的样本点集中在决策边界的附近， 因此许多样本点的置信度靠近0.5左右，即使决策边界能够将样本点判断正确，模型本身对这个结果也不是非常的新人
# 离边界远的点的置信度会很高 ，因为大概率不会被判断错误，所以支持向量机面对混合度较高的数据时，有着天生的置信度不足的缺点



In [ ]:
# 绘制直方图来查看预测概率的分布
# 这里的分箱是为了把概率分成一个个区间，可靠性曲线中的分箱是为了曲线的平滑
fig, ax2 = plt.subplots(figsize=(8, 6))
for clf, name_ in zip([gnb, logi, svc], name):
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_valid)
    # hasattr(obj.name): 查看一个类obj中是否存在名字为name的接口，存在则返回true
    if hasattr(clf, "predict_proba"):
        prob_pos = clf.predict_proba(x_valid)[:, 1]
    else: # use decision function 
        prob_pos = clf.decision_function(x_valid)
        prob_pos = (prob_pos - prob_pos.min()) / (prob_pos.max() - prob_pos.min())
    ax2.hist(prob_pos, bins=10, label=name_, histtype="step", lw=2)
    # histtype设置直方图为透明，lw是直方图每个柱子的粗细
    # 预测概率直接当x，y自己计算属于x区间的取值有多少
ax2.set_xticks(list(np.arange(0, 1, 0.1)))
ax2.legend()
fig.show()

# 支持向量机过于自卑， 朴素贝叶斯过于自负， 只有逻辑回归正好


In [ ]:
# 校准可靠性曲线 ， 有两种回归的方法， 原理可以自己去查，这些类只有predict_proba接口调用
# 参数输入已经训练完的模型
#cv默认5折，对于输入整数或者是none，二分类就是自动使用sklearn.model_selection_stratifiedKFold进行分割
# y如果是连续性变量，就使用sklearn.medel_selection.KFold进行分割
# 或者输入其他类建好的交叉验证模式或者生成器cv
# 或者可迭代的， 已经分割好的训练集和测试集索引数组
# cv=prefit,则假设已经在分类器上拟合完毕数据， 这种模式下， 使用者要手动确定用来拟合分类器的数据和即将被校准的数据没有交集
# method sigmoid 使用基于platt的sigmoid模型来进行校准 或者isotonic使用等渗回归进行校准（倾向过拟合)
name = ["gaussianbayes", 'logistic', 'bayes_isotonic', 'bayes+signoid']
gnb = GaussianNB()
models = [gnb, LR(C=.1, solver="lbfgs", max_iter=3000, multi_class="auto"),
        CalibretedClassifierCV(gnb, cv=2, method="isotonic"), 
        CalibretedClassifierCV(gnb, cv=2, method="sigmoid"), 
        ]
# plot_calib是把之前的画图函数给集成了一个函数，然后自动画图
plot_calib(models, name, x_train, xvalid, y_train, y_valid)
# 结果是等渗修正的比较好， sigmoid修正的稍微差一点
# 这时再看一下模型的精确性有没有提升


In [ ]:
from sklearn.calibration import CalibratedClassifierCV
gnb = GaussianNB().fit(x_train, y_train) # 0.86
gnb.score(x_valid, y_valid) # 0.11
brier_score_loss(y_valid, gnb.predict_proba(x_valid)[:,1], pos_label=1)
gnbisotonic = CalibratedClassifierCV(gnb, cv=2, method="isotonic").fit(x_train, y_train)
gnbisotonic.score(x_valid, y_valid)#0.86
brier_score_loss(y_valid, gnbisotonic.predict_proba(x_valid)[:, 1],pos_label=1) # 0.09
# 可靠性曲线更靠谱布利尔下降了，准确率却降低了
# 解释一下：不如决策树和支持向量机，概率不是概率而是置信度，而且这些模型的分类也不是靠概率而是靠决策边界，可能存在这类别1下的概率为0.4但样本依旧被分成1的情况，这种情况下模型没信心把这个样本分成1但还是分成1了
# 这个时候进行概率校准可能会想着更加错误的方向调整，比如0.4调整更接近0，模型直接判断成0了， 最终判断错误，所以布利尔和精确性相反的趋势
# 对于朴素贝叶斯磊说， 我们的贝叶斯时有偏估计，校准后，模型的预测更加接近真实概率，但是一组数据集上可能准确路下降，这取决于我们的测试集有多贴近我们估计的真实样本面貌
# 当两者相悖时，以准确率为准，这不是说布利尔指数就无效了，因为概率类模型几乎没有参数调整，我们可以用布利尔指数作为调整，进行概率校准， 指不定就会结果更好



In [ ]:
# 多项式朴素beiyesi
# 比如投硬币就是多项式中的二项分布，掷色子就是多项式分布
# 多项式分布擅长分布型变量
# 现实生活中，我们使用高斯贝叶斯处理连续性变量
# 多项式实验中的实验结果都很具体， 特征往往是次数频率计数出现这样的概念，多是离散的正整数，因此不接受负值的输入
#因为以上的特点，所以多项式的特征矩阵大多是稀疏矩阵，经常用于文本分类，使用TF-IDF向量技术，也可以使用简单的单词计数向量配合贝叶斯使用
# 多项式的参数alpha:a越大，精确率越低， 布利尔分数越高， 因为平滑选项，会增加噪音
# fit_prior时候学习先验概率，不给出就统一先验概率，为相同的概率大小，比如投色子掷硬币先验就是一样的
# calss_prior 表示类的先验概率，就是fit_prior为none， 自己提供先验
from sklearn.preprocessing import MinMaxScaler
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_blobs
from sklearn.metrics import brier_score_loss



In [ ]:
# 构造数据集
class_1 = 500
class_2 = 500
centers = [[0.0, 0.0], [2.0, 2.0]] # 设定两个类别的中心
clusters_std = [0.5, 0.5] # 设定两个类别的方差
X, y = make_blobs(n_samples=[class_1, class_2],
                centers=centers,
                cluster_std=clusters_std,
                random_state=0, shuffle=False)
X.shape

x_train, x_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state=420)




In [ ]:
import numpy as np
mms = MinMaxScaler().fit(x_train)# 在训练集上实例化训练我们的模型,保证没有负值
x_train_ = mms.transform(x_train)
x_vaild_ = mms.transform(x_vaild)
mnb = MultinomialNB().fit(x_train_, y_train)
# 重要属性，调用根据数据获取的， 每个标签类的对数先验概率log（P（Y））
# 由于概率永远是在01之间，所以先验概率永远是负数
mnb.class_log_prior_ # 先验概率非常的相似，所以应该是对半分
(y_train == 1).sum() / y_train.shape[0]
# 验证确实是对半分
mnb.class_log_prior_.shape # 永远等于标签中所带的类别数量
# 可以使用np.exp查看真正的概率
np.exp(mnb.class_log_prior_)
# 重要属性：返回一个固定标签类别下的每个特征的对数概率log(P(Xi|y))
mnb.feature_log_prob_ # 每个特征，每个标签
# 重要属性：在fit时每个标签类别下包含的样本数
# 当fit接口中的sample_weight被设置时,该接口返回的值也会受到加权的影响
mnb.class_count_
#predict predict_proba 都有
# 多项式适合离散的数据，现在时连续的，有几种操作，把x_train转换成分类数据，x_trian没有归一化因为哑变量之后没有负数了
from sklearn.preprocessing import KBinsDiscretizer # 对连续性的数据进行分箱
kbs = KBinsDiscretizer(n_bins=10, encode="onehot").fit(x_train)
x_train_ = kbs.transform(x_train)
x_valid_ = kbs.transform(x_valid)
x_train_.shape # (700, 20) 2个特征中，每个特征分了10个箱所分出来的哑变量
mnb = MultinomialNB().fit(x_train_, y_train)
# 分类的结果非常好,所以对分类型的数据效果非常好
mnb.score(x_valid_, y_valid)
# 伯努利朴素贝叶斯， 服从多元的二项分布
#这个类要求将样本转化成二分类特征向量,若不是二分类，就用binarize进行转化
# 伯努利更加关注的是是否存在，而多项式贝叶斯更关注出现的次数和频率。因此文本分类中，对于单词出现向量（而不是单词次数向量）训练分类器效果更好
# 而且在文档较短的数据集上，效果更好
# 参数 alpha binarize设定将特征二值化的阈值，若为none，则二值化默认完成


from sklearn.naive_bayes import BernoulliNB

mms = MinMaxScaler().fit(x_train)# 在训练集上实例化训练我们的模型,保证没有负值
x_train_ = mms.transform(x_train)
x_vaild_ = mms.transform(x_vaild)

# 不设置二值化
bn1_ = BernoulliNB().fit(x_train_, y_train)
#bn1_.score(x_valid_, y_valid)
brier_score_loss(y_valid, bn1_.predict_proba(x_valid_)[:, 1], pos_label=1)
#设置二值化
bn1 = BernoulliNB(binarize=0.5).fit(x_train_, y_train)

bn1.score(x_valid_, y_valid)
brier_score_loss(y_valid, bn1.predict_log_proba(x_valid_)[:, 1],pos_label=1)



In [ ]:
# 样本不均衡问题
from sklearn.metrics import brier_score_loss as BS, recall_score, roc_auc_score as AUC
class_1 = 50000
class_2 = 500
centers = [[0.0, 0.0],[5.0, 5.0]]
clusters_std = [3, 1]
X, y = make_blobs(n_samples=[class_1, class_2], centers=centers, cluster_std=clusters_std,random_state=0, shuffle=False)

name = ["Multinomial", "Gaussian","Bernoulli"]
models = [MultinomialNB(), GaussianNB(), BernoulliNB()]
# 适合分类型，任意型， 二分类型

In [ ]:
for name, clf in zip(name, models):
    x_train, x_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state=420)
    # 数据预处理
    if name != "Gaussian":
        kbs = KBinsDiscretizer(n_bins=10, encode="onehot").fit(x_train)
        x_train = kbs.transform(x_train)
        x_valid = kbs.transform(x_valid)
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_valid)
    proba = clf.predict(x_valid)
    score = clf.score(x_valid, y_valid) # 准确率
    print(name)
    print("\tbrier:{:.3f}".format(BS(y_valid,proba,pos_label=1)))
    print("\taccuracy:{:.3f}".format(score))
    print("\trecall:{:.3f}".format(recall_score(y_valid, y_pred)))
    print("\tauc:{:.3f}".format(AUC(y_valid, proba)))
    # 多项式完全放弃了少数类
    # 高斯都分别判断错了一点，但是找少数类还不如投硬币
    # 伯努利找少数类还行
    # 这时候怎么做才能对少数类进行处理

In [ ]:
# 文本分析
# 首先介绍单词计数向量技术
sample = ["Mashin learning is fascinating, it is wonderful",
            "Machine learning is a sensational technology",
            "Elsa is a popular character"]
            

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer()
X = vec.fit_transform(sample)
X # 得到了三行十一列的稀疏矩阵，三个样本，11个单词
vec.get_feature_names()# 获得每个列的名字
#X是特征矩阵，vec是模型训练的结果
import pandas as pd
# 稀疏矩阵无法输入pandas
CVresult = pd.DataFrame(X.toarray(), columns= vec.get_feature_names())
CVresult

In [ ]:
# 问题： 第一个是句子本身长会对多项式贝叶斯的分子有较大的权重
# 极端情况，一个句子几万个，其他的几个单词，那少单词的句子就完全被忽略了
# 有 l2 范式就是消除了上面这个影响
# is出现的多，但是没什么语义的影响
#is 没有什么意义但是权重还很大，所以改进TF-IDF
# 这个算法是一个单词月常见，他的权重就会越小，用这个方式压制频繁出现的无意义的额次
from sklearn.feature_extraction.text import   TfidfVectorizer as TFIDF

In [ ]:
vec = TFIDF()
X = vec.fit_transform(sample)
TFIDFresult = pd.DataFrame(X.toarray(), columns=vec.get_feature_names())
TFIDFresult


CVresult.sum(axis=0) # 返回每一列的和，11 个结果

CVresult.sum(axis=0) / CVresult.sum(axis=0).sum() # 得到每个特征向量的权重

TFIDFresult.sum(axis=0) / TFIDFresult.sum(axis=0).sum()

# 将原本出现次数比较多的词，压缩了相对应的权重


In [ ]:
# 开始探索文本数据
from sklearn.datasets import fetch_20newsgroups
data = fetch_20newsgroups()

In [ ]:
data.target_names # 不同类型的新闻
#里面数据巨大，表示一个类，里面有可调用的参数，我们先查看相应的参数有什么，来帮助我们解题

#里面有训练集和测试集
# 从categories里面选择想要的新闻类别shuffle表述是否打乱样本顺序
# 比如随机梯度下降需要随机性
import numpy as np
import pandas as pd
categories = ["sci.space",'rec.sport.hockey','talk.politics.guns','talk.politics.mideast'
    
]
train = fetch_20newsgroups(subset="train", categories=categories)
test = fetch_20newsgroups(subset="test", categories=categories)

In [ ]:
train.target_names
len(train.data) # 2303篇文章
train.data[0]
np.unique(train.target) # 总共有四个类标签
# 查看是否有不平衡的问题,没有样本不均衡的问题
for i in [1,2,3]:
    print(i, (train.target == i).sum()/len(train.target))

In [ ]:
x_train = train.data
x_valid = train.data
y_train = train.target
y_valid = train.target
tfidf = TFIDF().fit(x_train)
x_train_ = tfidf.transform(x_train)
x_valid_ = tfidf.transform(x_valid)

In [ ]:
x_train_ # 稀疏矩阵

In [ ]:
tosee = pd.DataFrame(x_train_.toarray(), columns=tfidf.get_feature_names())
tosee.head()

In [ ]:
# 在不同的贝叶斯上进行，不适用高斯因为高斯不接受稀疏矩阵
from sklearn.naive_bayes import MultinomialNB, ComplementNB, BernoulliNB
from sklearn.metrics import brier_score_loss as BS
name = ["Multinomial","Complement","Bernoulli"]
models = [MultinomialNB(),ComplementNB(),BernoulliNB()]
for name, clf in zip(name,models):
    clf.fit(x_train_, y_train)
    y_pred = clf.predict(x_valid_)
    proba = clf.predict_proba(x_valid_)
    score = clf.score(x_valid_, y_valid)
    print(name)
    # 4个不同标签取值下的布利尔分数
    Bscore = []
    for i in range(len(np.unique(y_train))):
        bs = BS(y_valid, proba[:, i],pos_label=i)
        Bscore.append(bs)
        print("\tbrier under{}:{:.3f}".format(train.target_names[i],bs))
    print("\taverage brier:{:.3f}".format(np.mean(Bscore)))
    print("\taccuracy:{:.3f}".format(score))
    print("\n")
# 补集贝叶斯精确度最好，但是不是很稳定，就是罗卜尔指数比较大
# 二分类的不好， 多项式的还可以


In [ ]:
# 继续看看能不能进行调整进行校准
from sklearn.calibration import CalibratedClassifierCV
name1 = ["Multinomial","Multinomial+isotonic","Multinomial+sigmoid","Complement","Complement+isotonic",
"Complement+sigmoid","Bernoulli","Bernoulli+isotonic","Bernoulli+sigmoid"]
modelss = [MultinomialNB()
            ,CalibratedClassifierCV(MultinomialNB(), cv=2, method="isotonic")
            ,CalibratedClassifierCV(MultinomialNB(), cv=2, method="sigmoid")
            ,ComplementNB()
            ,CalibratedClassifierCV(ComplementNB(), cv=2, method="isotonic")
            ,CalibratedClassifierCV(ComplementNB(), cv=2, method="sigmoid")
            ,BernoulliNB()
            ,CalibratedClassifierCV(BernoulliNB(), cv=2, method="isotonic")
            ,CalibratedClassifierCV(BernoulliNB(), cv=2, method="sigmoid")]
for name, clf in zip(name1,modelss):   
    clf.fit(x_train_, y_train)
    y_pred = clf.predict(x_valid_)
    proba = clf.predict_proba(x_valid_)
    score = clf.score(x_valid_, y_valid)
    print(name)
    # 4个不同标签取值下的布利尔分数
    Bscore = []
    for i in range(len(np.unique(y_train))):
        bs = BS(y_valid, proba[:, i],pos_label=i)
        Bscore.append(bs)
        print("\tbrier under{}:{:.3f}".format(train.target_names[i],bs))
    print("\taverage brier:{:.3f}".format(np.mean(Bscore)))
    print("\taccuracy:{:.3f}".format(score))
    print("\n")
# 选择精确性最高的模型
#compltment+sigmoid